In [0]:
%sh
ls

# Download source

In [0]:
!pip install mlflow
!pip install gdown
!gdown 1A-Uwe762UZ8ufw1KpkeuFPYF7znvnVy4

    

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 34.6 MB 21 kB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.4-py3-none-any.whl size=142891 sha256=1676740057f1cf37ff856ea07df2d788a1b70edc30b78170b59f330e77a9e4cb
  Stored in directory: /root/.cache/pip/wheels/48/7c/6e/4bf2c1748c7ecf994ca951591de81674ed6bf633e1e337d873
Successfully built databricks-cli
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: packaging
    Found existing installation: packagi

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import numpy as np
from datetime import datetime

In [0]:
import tarfile
!mkdir yelp_dataset
my_tar = tarfile.open('yelp_dataset.tar')
my_tar.extractall('yelp_dataset')
my_tar.close()


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
<command-2771410092097464> in <module>
      1 import tarfile
      2 get_ipython().system('mkdir yelp_dataset')
----> 3 my_tar = tarfile.open('yelp_dataset.tar')
      4 my_tar.extractall('yelp_dataset')
      5 my_tar.close()

/usr/lib/python3.8/tarfile.py in open(cls, name, mode, fileobj, bufsize, **kwargs)
   1601                     saved_pos = fileobj.tell()
   1602                 try:
-> 1603                     return func(name, "r", fileobj, **kwargs)
   1604                 except (ReadError, CompressionError):
   1605                     if fileobj is not None:

/usr/lib/python3.8/tarfile.py in gzopen(cls, name, mode, fileobj, compresslevel, **kwargs)
   1665 
   1666         try:
-> 1667             fileobj = GzipFile(name, mode + "b", compresslevel, fileobj)
   1668         except OSError:
   1669             if fileobj is

In [0]:
%sh
cd yelp_dataset
ls

In [0]:
spark = SparkSession.builder\
            .appName ("Restaurant Recommendation System") \
            .getOrCreate()

In [0]:
df = spark.read.json("file:///databricks/driver/yelp_dataset/yelp_academic_dataset_review.json")
df.printSchema()
df.show()

# Sơn

In [0]:
from pyspark.sql.functions import isnan
df.where(isnan(col("user_id"))).show()

In [0]:
df.where(isnan(col("stars"))).show()

In [0]:
df.where(isnan(col("review_id"))).show()

In [0]:
df.where(isnan(col("date"))).show()

# Preprocessing Data

In [0]:
ratings=df.select("user_id","review_id","stars","date")

In [0]:
ratings.show()

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

userRatings=ratings.groupBy("user_id").count().withColumn("userid", monotonically_increasing_id().cast(IntegerType()))


In [0]:
print(userRatings)
userRatings.show()

In [0]:
busiRatings=ratings.groupBy("business_id").count().withColumn("businessid", monotonically_increasing_id().cast(IntegerType()))

newratings=ratings.join(userRatings, ['userid'])
newratings=newratings.join(busiRatings, ['businessid'])
newratings=newratings.select("userid","businessid","stars","date")


In [0]:
print(ratings.count())
print(newratings.count())

In [0]:
print(newratings)
newratings.show()

# Tuấn

## Split data by special way

In [0]:
## Normal
# (training, testing) = newratings.randomSplit([0.8, 0.2])

In [0]:
## orther date
num=int(newratings.count()*0.1)
ortherDate=newratings.orderBy(col("date").desc())
testing=ortherDate.limit(int(num))
training=ortherDate.subtract(testing)

# Training Data

In [0]:
start = datetime.now()

als= ALS(maxIter=5, regParam=0.01, userCol="userid", itemCol="itemid", ratingCol="stars")
model = als.fit(training)
model.setColdStartStrategy("drop")
end = datetime.now()
print(f"Excute time: {end-start}")

# Prediction

In [0]:
predictions=model.transform(testing)
predictions.show()

# Validation

In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Mạnh

## This is multi threading, but it seem useless on single cluster

In [0]:
from multiprocessing.pool import ThreadPool as Pool

def worker(i):
    als= ALS(maxIter=i, regParam=0.01, userCol="userid", itemCol="itemid", ratingCol="stars")
    model = als.fit(training)
    model.setColdStartStrategy("drop")
    predictions=model.transform(testing)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    print(f"time{i}: {rmse}")
    
pool = Pool(10)
for item in range(5,15):
    pool.apply_async(worker, (item,))
    
pool.close()
pool.join()

In [0]:

5.821110777822227

itermax=10

5.4301693811058005
itermax=5
5.4301693811058005

# Cross validation

In [0]:
# Cross validation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

als= ALS(rank=5, maxIter=5, regParam=0.15, userCol="userid", itemCol="businessid", ratingCol="stars", nonnegative=True,implicitPrefs = False,coldStartStrategy="drop")
print("Load complete als model")
print(als)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
params = ParamGridBuilder().addGrid(als.regParam,[.01, .05, .1, .15]).build()
print(params)
print("That's params")
cv = CrossValidator(estimator=als, estimatorParamMaps=params, evaluator=evaluator, parallelism=4, numFolds= 5)
print(f"Load cross validation {cv}")
best_model = cv.fit(newratings)
model = best_model.bestModel
print(model)

In [0]:
predictions=model.transform(testing)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"RMSE={rmse}")

## we see the model could be depend on data, so we try with stratified sampling. This is data

In [0]:
gre_histogram = newratings.select('stars').rdd.flatMap(lambda x: x).histogram(11)

pd.DataFrame(
    list(zip(*gre_histogram)), 
    columns=['bin', 'frequency']
).set_index(
    'bin'
).plot(kind='bar');

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-616498814848837> in <module>
----> 1 gre_histogram = newratings.select('stars').rdd.flatMap(lambda x: x).histogram(11)
      2 
      3 pd.DataFrame(
      4     list(zip(*gre_histogram)),
      5     columns=['bin', 'frequency']

NameError: name 'newratings' is not defined

# Đạt

# The problem is identity monotonically_increasing_id

### this just convert 10% of userid to int

In [0]:
print(userRatings.count())
print(busiRatings.count())


In [0]:
# So replace to:

userRatings=ratings.groupBy("user_id").count()
## take id type int
window = Window.orderBy(col('user_id'))
userRatings = userRatings.withColumn('userid', row_number().over(window))

# Another way:

## we collect by get double new data on train to test, then just take haft part by random

# New way

In [0]:
def SplitTest(source,data):
    train,test=data.randomSplit([0.3,0.7])
    try:
        return source.union(train)
    except:
        return train

# For each new user, we random to collect 30%, but it take so long time

In [0]:
## Keep data test with keep
newTrain = None
userNew=new.groupBy('userid').count().select('userid')
lstuserNew=userNew.rdd.flatMap(lambda x: x).collect()

for i in lstuserNew:
    userI=new.filter(col('userid')==i)
    newTrain=SplitTest(newTrain,userI)
    

## show

In [0]:
# Show

rcm = model.recommendForAllUsers(10)
rcm.show(10)

In [0]:
recommendations = rcm.withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("newratings.businessid"), col("newratings.stars"))
recommendations.limit(10).show()